## Recommender System - Marketplace Matching

In this notebook, we will: 
- Clean textual data from user-input verbatim posts
- Use a Word2Vec model to calculate document similarities 
- Sort the most similar user input to our training data in order to recommend similar products 
- Save this model in a format that allows us to refresh the testing data

The goal of this project is to create a recommender system to help Pangeans find the right "project" for them, given their profile information. Here, we are using legacy data that is from Pangea V2, when Pangeans were allowed to post services and requests, as well as purchase items on the platform. We are using the User-Inputted Titles to suggest similar services, or in V3, similar "projects". 

In [16]:
#Importing Libraries
import numpy as np
import pandas as pd

import sys
from pandas import DataFrame

import json
from pandas.io.json import json_normalize
import csv

import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from gensim import corpora
from collections import defaultdict
from pprint import pprint

from gensim.models.word2vec import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts

import re
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords') ###

import gensim 
from gensim.models import KeyedVectors
import gensim.downloader as api

from operator import itemgetter, attrgetter

from gensim.models.doc2vec import Doc2Vec

import os, sys

from operator import add

from sklearn.preprocessing import normalize

import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from adjustText import adjust_text

from sklearn.manifold import TSNE

import pickle

import json
from pprint import pprint
from pandas.io.json import json_normalize

import operator

import csv

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/angelateng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def vectorize_and_store_existing_titles():
    
    raw = pd.read_csv("allPostData.csv", header=0);
    
    #we can replace this with a filepath in the future
    titles = raw['title'];
    post_titles = [title for title in titles];
    tokens = [[word for word in title.lower().split()] for title in post_titles];
    
    clean_words = [[word for word in title if word.isalpha()] for title in tokens];
    stoplist = set(stopwords.words('english'));
    
    titles_nostopwords = [[word for word in title if word not in stoplist] for title in clean_words];   
    
    model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True); 
    filtered_word_list = [[word for word in title if word in model.vocab] for title in titles_nostopwords];
    
    title_vectors = {}
    for title in filtered_word_list: 
        word_vecs = [model[word] for word in title]
        if len(word_vecs) == 0:
            title_vec = [np.zeros(300)]
        else: 
            title_vec = normalize(sum(word_vecs).reshape(1, -1))
        title_vectors[" ".join(title)] = title_vec[0]
    #do we want to print out the original titles? how to do this?
    vectorized_titles = pd.DataFrame.from_dict(title_vectors)
    
    #can also replace filepath in the future
    vectorized_titles.to_pickle("/Users/angelateng/Dropbox/SharpestMinds/vectorized_titles.pkl")
    return(vectorized_titles)

#vectorize_and_store_existing_titles()


In [3]:
#pd.read_pickle("/Users/angelateng/Dropbox/SharpestMinds/vectorized_titles.pkl")
#sanity check

In [3]:
#can change this directory later

with open('allPostData.json') as fresh_data:
    user_post = json.load(fresh_data)

json_df = pd.DataFrame.from_dict(json_normalize(user_post), orient='columns')

def vectorize_new_title(json_df):
    title = json_df["title"]
    
    json_post_titles = [title for title in title];
    json_tokens = [[word for word in title.lower().split()] for title in json_post_titles];
    
    json_clean_words = [[word for word in title if word.isalpha()] for title in json_tokens];
    
    stoplist = set(stopwords.words('english'));
    json_titles_nostopwords = [[word for word in title if word not in stoplist] for title in json_clean_words];   
    
    model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True); 
    json_filtered_word_list = [[word for word in title if word in model.vocab] for title in json_titles_nostopwords];
    
    json_title_vectors = {}
    for title in json_filtered_word_list: 
        json_word_vecs = [model[word] for word in title]
        if len(json_word_vecs) == 0:
            json_title_vec = [np.zeros(300)]
        else: 
            json_title_vec = normalize(sum(json_word_vecs).reshape(1, -1))
        json_title_vectors[" ".join(title)] = json_title_vec[0]
    #do we want to print out the original titles? how to do this?
    json_vectorized_titles = pd.DataFrame.from_dict(json_title_vectors)

    return(json_vectorized_titles)

#vectorize_new_title(json_df)

In [6]:
vectorized_title = vectorize_new_title(json_df)

def rank_existing_titles(vectorized_title):
    #loop over all keys in dict 
    ranked_titles = {}
    other_titles = pd.read_pickle("/Users/angelateng/Dropbox/SharpestMinds/vectorized_titles.pkl")
    #can also use title_vectors.keys() 
    for key in vectorized_title:
        ranked_titles[key] = np.dot(other_titles[key], vectorized_title[key])
    sorted_title_vecs = sorted(ranked_titles.items(), key=operator.itemgetter(1), reverse=True)
    #print(sorted_title_vecs)
    return(sorted_title_vecs)

#rank_existing_titles(vectorized_title[:10])

[('ride boston', 1.0000004),
 ('league legends booster', 1.0000004),
 ('video making lessons', 1.0000004),
 ('fix bike', 1.0000004),
 ('tattoo design', 1.0000004),
 ('web development design', 1.0000004),
 ('personal shopper', 1.0000004),
 ('startup opportunity', 1.0000004),
 ('spiritual talks', 1.0000004),
 ('fashion designer', 1.0000004),
 ('looking team member', 1.0000004),
 ('matcha buddy', 1.0000004),
 ('pogo walker', 1.0000004),
 ('camp counselor', 1.0000004),
 ('long boarding lessons', 1.0000002),
 ('resume cover letter', 1.0000002),
 ('guitar lessons', 1.0000002),
 ('gym buddy', 1.0000002),
 ('snowboarding lessons', 1.0000002),
 ('get startup job', 1.0000002),
 ('orgo kit', 1.0000002),
 ('cs tutor', 1.0000002),
 ('ethernet cable', 1.0000002),
 ('soccer lessons', 1.0000002),
 ('idea help', 1.0000002),
 ('programming advise', 1.0000002),
 ('workout plan', 1.0000002),
 ('piano lessons', 1.0000002),
 ('latex tutor', 1.0000002),
 ('reviews', 1.0000002),
 ('beginner chess lessons', 1.

In [35]:
def generate_recommendations(json_df):
    title = json_df["title"]
    vectorized_title = vectorize_new_title(json_df)
    ranked_titles = rank_existing_titles(vectorized_title)
    
    with open("/Users/angelateng/Dropbox/SharpestMinds/ranked_titles.csv", "w", newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        for title in ranked_titles:
            wr.writerow([ranked_titles, title])
    #csv_out = ranked_titles.to_csv('ranked_titles', encoding='utf-8', index=False)
    #writer = csv.writer(ranked_titles, delimiter='', quotechar='"', quoting=csv.QUOTE_ALL)
 
    #for row in reader:
        #writer.writerow(row)

       # with open('csv_out', 'wb') as csv_out:
         #   csv_writer = csv.writer(csv_out, quoting=csv.QUOTE_ALL)
         #   csv_writer.writerow('csv_out')
    return(ranked_titles)
    
    

#vectorize_and_store_existing_titles(); 
#vectorize_new_title(json_df);
#rank_existing_titles(vectorize_new_title);

In [36]:
generate_recommendations(json_df)

[('ride boston', 1.0000004),
 ('league legends booster', 1.0000004),
 ('video making lessons', 1.0000004),
 ('fix bike', 1.0000004),
 ('tattoo design', 1.0000004),
 ('web development design', 1.0000004),
 ('personal shopper', 1.0000004),
 ('startup opportunity', 1.0000004),
 ('spiritual talks', 1.0000004),
 ('fashion designer', 1.0000004),
 ('looking team member', 1.0000004),
 ('matcha buddy', 1.0000004),
 ('pogo walker', 1.0000004),
 ('camp counselor', 1.0000004),
 ('long boarding lessons', 1.0000002),
 ('resume cover letter', 1.0000002),
 ('guitar lessons', 1.0000002),
 ('gym buddy', 1.0000002),
 ('snowboarding lessons', 1.0000002),
 ('get startup job', 1.0000002),
 ('orgo kit', 1.0000002),
 ('cs tutor', 1.0000002),
 ('ethernet cable', 1.0000002),
 ('soccer lessons', 1.0000002),
 ('idea help', 1.0000002),
 ('programming advise', 1.0000002),
 ('workout plan', 1.0000002),
 ('piano lessons', 1.0000002),
 ('latex tutor', 1.0000002),
 ('reviews', 1.0000002),
 ('beginner chess lessons', 1.